# IBM - Applied Data Science Capstone 

## Capstone Project - The Battle of Neighbourhoods

Opening a New Shopping Mall in Bengaluru, Karnataka, India

1. Build a dataframe of neighborhoods in Bengaluru  by web scraping the data from Wikipedia page.
2. Get the geographical coordinates of the neighborhoods using Python  Geocoder package.
3. Obtain the venue data for the neighborhoods from Foursquare API.
4. Explore and cluster the neighborhoods.
5. Visualize the data and derive insights.
6. Select the best cluster to open a new shopping mall.

## 1.0 INTRODUCTION

The city of Bengaluru, in Karnataka is called as the Silicon Valley of India. This city is flooded with IT and Tech companies which boosts the GDP per capita of the city marking its place at fifth position in India [1]. Therefore, the buying potential of individuals is growing rapidly in this city.

For individuals, visiting shopping malls are like a one-stop destination for satisfying all their needs such as grocery shopping, dine at restaurants, fashion outlets, play games and watch movies. Retailers and Property developers are taking advantage of this trend to build more shopping malls to balance and meet the demand and supply.

As a result, there are many shopping malls in Bengaluru and many under construction. Yet, the demand of the consumers is not meet. Therefore, property developers are willing to construct more shopping malls in this city as this gives them a constant income. So now the property developers are left with a simple question - "Where should we construct a mall?". The is an important business decision to be taken as the location of the property decides its success or a failure.

## 2.0 METHODOLOGY

For the Capstone project I have analysed and selected the best location for construction of a mall in the Bengaluru city, Karnataka. This being a Data science project the methodology followed is,
1. Understanding the business problem
2. Data collection
3. Data cleaning
4. Modelling 
5. Data Visualization
6. Business insights
7. Data Driven decision

Make API calls to Foursquare passing in the geographical coordinates of the neighbourhoods in a Python loop. Foursquare will return the venue data in JSON format and we will extract the venue name, venue category, venue latitude and longitude. With the data, we can check how many venues were returned for each neighbourhood and examine how many unique categories can be curated from all the returned venues. 
Then, we will analyse each neighbourhood by grouping the rows by neighbourhood and taking the mean of the frequency of occurrence of each venue category. By doing so, we are also preparing the data for use in clustering. Since we are analysing the “Shopping Mall” data, we will filter the “Shopping Mall” as venue category for the neighbourhoods.

Lastly, we will perform clustering on the data by using k-means clustering. K-means clustering algorithm identifies k number of centroids, and then allocates every data point to the nearest cluster, while keeping the centroids as small as possible. It is one of the simplest and popular unsupervised machine learning algorithms and is particularly suited to solve the problem for this project. We will cluster the neighbourhoods into 3 clusters based on their frequency of occurrence for “Shopping Mall”. 
The results will allow us to identify which neighbourhoods have higher concentration of shopping malls while which neighbourhoods have fewer number of shopping malls. Based on the occurrence of shopping malls in different neighbourhoods, it will help us to answer the question as to which neighbourhoods are most suitable to open new shopping malls.



In [1]:
# Library to handle data in a vectorized manner
import numpy as np 

# Library for data analsysis
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Library to handle JSON files
import json 


# Convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
import geocoder 


# Library to handle requests
import requests

# library to parse HTML and XML documents
from bs4 import BeautifulSoup 

# Tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

# Map rendering library
import folium 

print("Libraries imported.")

Libraries imported.


# 3.0 DATA COLLECTION
### The data regarding neighbourhoods in Bengaluru is scrapped from wikipedia


<br>•  Latitude and longitude coordinates of those neighbourhoods. This is required in order to plot the map. 
<br>•  Venue data, particularly data related to shopping malls. We will use this data to perform clustering on the neighbourhoods. 
<br>•  The geographical coordinates are collected using Python Geocoder package 
<br>•  The details for the mall location is scrapped from the FourSquare developers website. 
 
The data required for this project is scrapped the the following sources (links), 
<br> 1.https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore
<br> 2.https://foursquare.com/explore?ll=10.466705%2C77.533295&mode=url&near=Bangalore%2C%20Karn%C4%81taka&q=Shopping%20Mall
  the following attributes are collected from this website,
    2.1 Neighbourhood 
    2.2 Latitude
    2.3 Longitude
    2.4 Venue Name 
    2.5 Venue Latitude 
    2.6 Venue Longitude 
    2.6 Venue Category

We will use web scraping techniques to extract the data from the Wikipedia page, with the help of Python requests and beautifulsoup packages. Then we will get the geographical coordinates of the neighbourhoods using Python Geocoder package which will give us the latitude and longitude coordinates of the neighbourhoods. After that, we will use Foursquare API to get the venue data for those neighbourhoods.


In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []

In [3]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [4]:
bl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

# 4.0 Data cleaning

Data cleansing or data cleaning is the process of detecting and correcting corrupt or inaccurate records from a record set, table, or database and refers to identifying incomplete, incorrect, inaccurate or irrelevant parts of the data and then replacing, modifying, or deleting the dirty or coarse data.

In [5]:
tr=bl_df.drop([0,1,2],inplace=False)
tr.head()

,Neighborhood
3,Adugodi
4,"Agara, Bangalore"
5,Ananthnagar
6,Anjanapura
7,Arekere


# 5.0 Define a function to get coordinates

In [6]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bengaluru, Karnataka, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
coords = [ get_latlng(neighborhood) for neighborhood in bl_df["Neighborhood"].tolist() ]

# A temporary dataframe to populate the coordinates into Latitude and Longitude

In [10]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# Merge the coordinates 

In [11]:
tr['Latitude'] = df_coords['Latitude']
tr['Longitude'] = df_coords['Longitude']

# Check the neighborhoods and its coordinates

In [12]:
print(tr.shape)

(128, 3)


In [13]:
# Save the DataFrame as CSV file
tr.to_csv("bl_df.csv", index=False)

# Get the coordinates of Bengaluru, Karnataka, India

In [14]:
address = 'Bengaluru, Karnataka, India '

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bengaluru, Karnataka, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bengaluru, Karnataka, India 12.9791198, 77.5912997.


# 6.0 Map of Bengaluru 

Folium makes it easy to visualize data that’s been manipulated in Python on an interactive Leaflet map. It enables both the binding of data to a map for choropleth visualizations as well as passing Vincent/Vega visualizations as markers on the map.

In [15]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for lat, lng, neighborhood in zip(tr['Latitude'], tr['Longitude'], tr['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [16]:
# save the map as HTML file
map_kl.save('map_bl_bang.html')

# 7.0 Foursquare Credentials and Version

In [17]:
CLIENT_ID = 'RG4H2GV5W3KMNXIQZCWVHSMF2GWLVAVJV2NV0JUEMVM21P4G' 
CLIENT_SECRET = '1QBUYAR5IEGETXA2VCVGD2ZUDVF0H4I2BWW0LUMTSKZYJI2O' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RG4H2GV5W3KMNXIQZCWVHSMF2GWLVAVJV2NV0JUEMVM21P4G
CLIENT_SECRET:1QBUYAR5IEGETXA2VCVGD2ZUDVF0H4I2BWW0LUMTSKZYJI2O


In [18]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(tr['Latitude'], tr['Longitude'], tr['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [19]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)

(6702, 7)


In [20]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,100,100,100,100,100,100
Ananthnagar,27,27,27,27,27,27
Anjanapura,6,6,6,6,6,6
Arekere,78,78,78,78,78,78
Austin Town,100,100,100,100,100,100
BTM Layout,86,86,86,86,86,86
Babusapalya,52,52,52,52,52,52
"Bagalur, Bangalore Urban",12,12,12,12,12,12
Bahubalinagar,24,24,24,24,24,24


In [21]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 237 uniques categories.


# List of unique categories

In [22]:
venues_df['VenueCategory'].unique()[0:10]

array(['Clothing Store', 'Movie Theater', 'Creperie', 'Burger Joint',
       'Indian Restaurant', 'Multiplex', 'Financial or Legal Service',
       'Café', 'Brewery', 'Breakfast Spot'], dtype=object)

In [23]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

# 8.0 One hot encoding

In [24]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

In [25]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(127, 238)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Booth,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adugodi,0.000000,0.000000,0.00000,0.000000,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.010000,0.000000,0.030000,0.01,0.00000,0.000000,0.000000,0.01,0.0000,0.010000,0.00,0.000000,0.00,0.000000

In [26]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

60

In [27]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [28]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Adugodi,0.010000
1,Ananthnagar,0.000000
2,Anjanapura,0.000000
3,Arekere,0.025641
4,Austin Town,0.010000


# 9.0 k-means clustering

The k-means clustering algorithm is an iterative algorithm that tries to partition the dataset into K pre-defined distinct non-overlapping subgroups (clusters) where each data point belongs to only one group.

The way kmeans algorithm works is as follows:

1. Specify number of clusters K.
2. Initialize centroids by first shuffling the dataset and then randomly selecting K data points for the centroids without replacement.
3. Keep iterating until there is no change to the centroids. i.e assignment of data points to clusters isn’t changing.

In [29]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1])

In [30]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [31]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Adugodi,0.010000,1
1,Ananthnagar,0.000000,1
2,Anjanapura,0.000000,1
3,Arekere,0.025641,0
4,Austin Town,0.010000,1


In [32]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(tr.set_index("Neighborhood"), on="Neighborhood")

# check the last columns!
print(kl_merged.shape)
kl_merged.head() 

(127, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adugodi,0.010000,1,12.94402,77.60800
1,Ananthnagar,0.000000,1,12.95408,77.54135
2,Anjanapura,0.000000,1,12.85811,77.55909
3,Arekere,0.025641,0,12.88568,77.59668
4,Austin Town,0.010000,1,12.96348,77.61297


In [33]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(127, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
63,Kammanahalli,0.032258,0,13.014300,77.636850
26,Cooke Town,0.027778,0,13.003630,77.622830
27,"Cox Town, Bangalore",0.027778,0,12.995940,77.626440
104,"Shivajinagar, Bangalore",0.020000,0,12.987200,77.604010
39,Gowdanapalya,0.040816,0,12.909680,77.556750
89,Peenya,0.043478,0,13.031850,77.526790
45,Hoodi,0.022222,0,12.991980,77.715060
47,Hulimavu,0.017241,0,12.880640,77.601470
48,Immadihalli,0.026316,0,12.966860,77.761580
87,Padmanabhanagar,0.028571,0,12.918140,77.559400


# 10.0 Create map of the clusters

Folium makes it easy to visualize data that’s been manipulated in Python on an interactive Leaflet map. It enables both the binding of data to a map for choropleth visualizations as well as passing Vincent/Vega visualizations as markers on the map.

In [34]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
# save the map as HTML file
map_clusters.save('map_clusters_bang.html')

# 11.0 RESULTS 

The results from the k-means clustering show that we categorize the neighbourhoods into 3 
<br>Clusters are based on the frequency of occurrence for “Shopping Mall”: 

<br>Cluster 0 [Red]: Areas with moderate number of shopping malls 
<br>Cluster 1 [Purple]: Areas with low number to no existence of shopping malls 
<br>Cluster 2 [Green]: Areas with high concentration of shopping malls

<img src="files/result.jpg">

# 12.0 CONCLUSIONS

1. From the map it can be seen that the Eastern and the North-Western parts of the city is flooded with malls
2. The count of malls in the southern part of the city is less.
3. This part of Bengaluru being newly developed is the optimal location for building a new Shopping Mall
4. The optimal areas include, 
    <br> 4.1 Electronic city phase - 1 
    <br> 4.2 Electronic city phase - 2
    <br> 4.3 Anekal
    <br> 4.4 Jigani
    <br> 4.5 Surya Nagar phase - 1
    <br> 4.6 Bommasandra
    
<br>This represents a great opportunity and high potential areas to open new shopping malls as there is no competition.